In [14]:
import pandas as pd
import numpy as np
import json
import gzip
from typesense import Client
from tqdm.auto import tqdm
from pathlib import Path
from sentence_transformers import SentenceTransformer

2023-06-10 17:59:41.207014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 17:59:41.435098: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-10 17:59:42.059824: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-10 17:59:42.059887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [6]:
data_dir = Path('/home/jameslittiebrant/DataspellProjects/hybridSearch/data/small-all-news/')
csvs = sorted(data_dir.glob('*.csv'))

In [8]:
df = pd.concat([pd.read_csv(file) for file in csvs])

In [9]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [11]:
df = df.drop('Unnamed: 0', axis=1)

In [15]:
client = Client({
    'nodes': [{
        'host': 'localhost', # For Typesense Cloud use xxx.a1.typesense.net
        'port': '8108',      # For Typesense Cloud use 443
        'protocol': 'http'   # For Typesense Cloud use https
    }],
    'api_key': 'xyz',
    'connection_timeout_seconds': 10
})

In [16]:
client.operations.is_healthy()

True

In [17]:
df.dtypes

id               int64
title           object
publication     object
author          object
date            object
year           float64
month          float64
url             object
content         object
dtype: object

In [19]:
schema = {
    'name': 'news',
    'fields': [
        {'name': 'id', 'type': 'int32' },
        {'name': 'title', 'type': 'string'},
        {'name': 'publication', 'type': 'string'},

        {'name': 'author', 'type': 'string' },
        {'name': 'date', 'type': 'string' },
        {'name': 'year', 'type': 'float' },
        {'name': 'month', 'type': 'float' },
        {'name': 'url', 'type': 'string' },
        {'name': 'content', 'type': 'string' },

        {'name': 'keyphrases', 'type': 'string[]' },
        {'name': 'people', 'type': 'string[]' },
        {'name': 'organizations', 'type': 'string[]' },
        {'name': 'locations', 'type': 'string[]' },
        {'name': 'vec', 'type': 'float[]', 'num_dim':384 }
    ]
}

In [20]:
client.collections.create(schema)

{'created_at': 1686434549,
 'default_sorting_field': '',
 'enable_nested_fields': False,
 'fields': [{'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'title',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'publication',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'author',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'date',
   'optional': False,
   'sort': False,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'year',
   'optional': False,
   'sort': True,
   'type': 'float'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'month',
   'optional': False,
   'sort

client.collections['news'].delete()

In [22]:
df['year'] = df['year'].fillna(1970)
df['month'] = df['month'].fillna(1)
df = df.fillna('')

In [23]:
df = df.to_dict(orient='records')

In [24]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [26]:
encoder.encode(df[0]['content']).tolist()

[-0.043169181793928146,
 0.09082550555467606,
 0.058354079723358154,
 -0.04722841829061508,
 0.014962717890739441,
 0.03772351145744324,
 0.030215851962566376,
 0.006577500142157078,
 0.023697124794125557,
 0.016201520338654518,
 -0.01949138380587101,
 0.09316234290599823,
 0.024865370243787766,
 -0.0639033168554306,
 -0.015670960769057274,
 0.02200692892074585,
 0.03845064342021942,
 -0.011750565841794014,
 -0.09076853096485138,
 0.1558375507593155,
 -0.043601471930742264,
 0.04033941030502319,
 -0.07877258956432343,
 0.022272925823926926,
 -0.012625121511518955,
 0.014968897216022015,
 -0.030939089134335518,
 -0.0744728147983551,
 -0.08111467212438583,
 -0.012672461569309235,
 0.1257838010787964,
 -0.07562428712844849,
 -0.026806272566318512,
 0.07473046332597733,
 0.012640764936804771,
 0.007119120564311743,
 -0.04998016729950905,
 -0.0029252765234559774,
 0.017073018476366997,
 0.010581588372588158,
 0.02321448363363743,
 -0.026614241302013397,
 -0.04922764003276825,
 0.05624394863

In [27]:
for _ in tqdm(df):
    _['keyphrases'] = []
    _['people'] = []
    _['organizations'] = []
    _['locations'] = []

  0%|          | 0/142570 [00:00<?, ?it/s]

In [31]:
vectors = list()
batch_size=128
for i in tqdm(range(0, len(df), batch_size)):
    _content = [x['content'] for x in df[i:i+batch_size]]
    _content = encoder.encode(_content).tolist()
    vectors.extend(_content)

  0%|          | 0/1114 [00:00<?, ?it/s]

In [32]:
len(df), len(vectors)

(142570, 142570)

In [33]:
for record, vec in zip(df, vectors):
    record['vec'] = vec

In [36]:
for document in tqdm(df):
    document['id'] = str(document['id'])
    client.collections['news'].documents.create(document)

  0%|          | 0/142570 [00:00<?, ?it/s]

In [43]:
search_parameters = {
    'q'         : 'abortion',
    'query_by'  : 'content',
    'exclude_fields': 'vec',
    'include_fields': 'publication,title,content'
}

results = client.collections['news'].documents.search(search_parameters)

In [44]:
len(results['hits'])

10

In [46]:
results['hits'][1]['document']

{'content': '     We’re now a month away from the inauguration of Donald Trump, and only a couple of weeks away from the   of a new Congress, one that at long last knows that it no longer has to worry that its bills will be vetoed by a president from the other party. As you might expect, many Republicans are giddy with excitement over all the new laws they’ll be able to pass, as they remake America into a paradise of liberty, prosperity and strong   values. But there’s danger lurking there, too, because a whole bunch of promises they’ve made in recent years are now going to have to be kept, and they won’t have a Democratic president to use as an excuse for avoiding ideologically important but politically perilous actions. For now I want to focus on just one of those actions: defunding Planned Parenthood. This is going to be one of the first items on the Republican agenda, not because the leadership necessarily relishes having a fight over it, but because Republican activists are alread

In [48]:
search_requests = {
    'searches': [
        {
            'collection': 'news',
            'q' : '*',
            'vector_query': f'vec:({encoder.encode("abortion").tolist()}, k:10)',
            'include_fields' : 'publication,title,content'
        }
    ]
}

In [49]:
common_search_params =  {}
vec_hits = client.multi_search.perform(search_requests, common_search_params)

In [56]:
vec_hits['results'][0]['hits'][0]['document']

{'content': ' Opponents of abortion often challenge   people with thought experiments about very late abortions. Rand Paul challenged Debbie Wasserman Schultz to affirm that she would be okay with ”killing a   baby that is not born yet.” Marco Rubio falsely claimed that Hillary Clinton even supports abortion ”on the due date.”    It’s easy for later abortions to be reduced to thought experiments  —   to be demonized and caricatured as something they’re not  —   because we rarely hear the stories of actual women who have had procedures like these. They’re incredibly rare, and doctors won’t perform them unless there’s a serious issue. So they are usually performed under deeply painful and difficult circumstances that aren’t easy to share. But Jezebel recently published a stunning interview with a woman who had an abortion at 32 weeks. It shows, in heartbreaking, human detail, exactly what might lead a woman to make a decision like this, and what the procedure is actually like. ”Elizabeth

In [57]:
import gzip
import json

In [59]:
with gzip.open('vectorized_small_news.json.gz', 'wt') as f:
    json.dump(df, f)